In [21]:
import pandas as pd
import numpy as np

In [22]:
data_completa = pd.read_csv('../DATOS/Honda.csv')

In [23]:
#LIMPIEZA DATOS
data_completa=data_completa.drop(columns=["Date","Open","High","Low","Adj Close","Volume"])

In [24]:
#PREPARAR DATOS POR RANGOS
numero_periodos=5
# Función para determinar el estado según el cambio
def determinar_estado(cambio):
    if cambio > -0.4 and cambio < 0.4 :
        return 'Estable'
    elif cambio < 0:
        return 'Bajo'
    else:
        return 'Subio'
# Calcula la diferencia entre los valores de cierre consecutivos y asigna el resultado a la columna 'Cambio'
data_completa['Cambio'] = data_completa['Close'].diff()
# Aplica la función 'determinar_estado' a la columna 'Cambio' y crea una nueva columna 'EstadoNuevo' con los resultados
data_completa['EstadoNuevo'] = data_completa['Cambio'].apply(determinar_estado)



#CSV pruebas para excel 
data_completa['EstadoNuevo'].to_csv('../DATOS/Honda_1.csv')
# Divide el DataFrame en un array de DataFrames, cada uno representando un periodo
data_por_periodo = np.array_split(data_completa,numero_periodos)




/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [25]:
frecuencias_observadas = []

for i in range(numero_periodos):
    # Calcula la tabla de frecuencias observadas usando crosstab entre el estado actual y el estado anterior (shift)
    frecuencia_obs = pd.crosstab(data_por_periodo[i]['EstadoNuevo'].shift(), data_por_periodo[i]['EstadoNuevo'], normalize=False)
    print("FRECUENCIA OBSERVADA - PERIODO", i+1, "\n", frecuencia_obs, "\n")
    # Agrega la frecuencia observada a la lista
    frecuencias_observadas.append(frecuencia_obs)


FRECUENCIA OBSERVADA - PERIODO 1 
 EstadoNuevo  Bajo  Estable  Subio
EstadoNuevo                      
Bajo            8        5      6
Estable         8        8      3
Subio           3        7      4 

FRECUENCIA OBSERVADA - PERIODO 2 
 EstadoNuevo  Bajo  Estable  Subio
EstadoNuevo                      
Bajo            4        5     10
Estable         6        4      4
Subio           9        4      5 

FRECUENCIA OBSERVADA - PERIODO 3 
 EstadoNuevo  Bajo  Estable  Subio
EstadoNuevo                      
Bajo            3        6      5
Estable         5       11      7
Subio           5        6      3 

FRECUENCIA OBSERVADA - PERIODO 4 
 EstadoNuevo  Bajo  Estable  Subio
EstadoNuevo                      
Bajo            5        5      3
Estable         5       11      8
Subio           3        8      3 

FRECUENCIA OBSERVADA - PERIODO 5 
 EstadoNuevo  Bajo  Estable  Subio
EstadoNuevo                      
Bajo            6        1      7
Estable         1        5      6
S

In [26]:
porcentaje_observado = [None] * numero_periodos

for i in range(0, numero_periodos):
    # Calcula los porcentajes observados dividiendo las frecuencias observadas por la suma total de cada periodo
    porcentaje_observado[i] = frecuencias_observadas[i] / frecuencias_observadas[i].sum()
    #Transpone la matriz para que sea más fácil de leer y comparar con el Excel
    porcentaje_observado[i] = porcentaje_observado[i].T
    
print("Porcentaje relativo:\n", porcentaje_observado[1]) #Ejemplo de impresion de un periodo


Porcentaje relativo:
 EstadoNuevo      Bajo   Estable     Subio
EstadoNuevo                              
Bajo         0.210526  0.315789  0.473684
Estable      0.384615  0.307692  0.307692
Subio        0.526316  0.210526  0.263158


In [27]:
# Calcula la tabla de frecuencias esperadas usando crosstab entre el estado actual y el estado anterior (shift) en todo el conjunto de datos
frecuencia_esperada= pd.crosstab(data_completa['EstadoNuevo'].shift(), data_completa['EstadoNuevo'])
print("PROBABILIDAD OBSERVADA \n",frecuencia_esperada,"\n")


PROBABILIDAD OBSERVADA 
 EstadoNuevo  Bajo  Estable  Subio
EstadoNuevo                      
Bajo           26       22     31
Estable        25       40     28
Subio          28       31     29 



In [28]:
# Calcula los porcentajes esperados dividiendo las frecuencias esperadas por la suma total de todas las frecuencias esperadas
porcentaje_esperado = frecuencia_esperada / frecuencia_esperada.sum()
#Transpone la matriz para que sea más fácil de leer y comparar con el Excel
porcentaje_esperado = porcentaje_esperado.T
print(porcentaje_esperado)


EstadoNuevo      Bajo   Estable     Subio
EstadoNuevo                              
Bajo         0.329114  0.316456  0.354430
Estable      0.236559  0.430108  0.333333
Subio        0.352273  0.318182  0.329545


In [35]:
ChiObs = 0
for i in range(0, numero_periodos): 
     # Suma acumulativa de los valores chi-cuadrado observados se aplica la formula vista en clase
    ChiObs+= np.sum(frecuencias_observadas[i].sum().values*(porcentaje_observado[i].values-porcentaje_esperado.values)**2/porcentaje_esperado.values)
    print(frecuencias_observadas[i].sum().values)
    print(porcentaje_observado[i].values)
    print(porcentaje_esperado.values)
    break
print(f'Chi Observado: {ChiObs}')


[19 20 13]
[[0.42105263 0.42105263 0.15789474]
 [0.25       0.4        0.35      ]
 [0.46153846 0.23076923 0.30769231]]
[[0.32911392 0.3164557  0.35443038]
 [0.23655914 0.43010753 0.33333333]
 [0.35227273 0.31818182 0.32954545]]
Chi Observado: 3.8067366927467106


In [30]:
from scipy.stats import chi2
grados_libertad_total = 0
sum_filas = 0
# Iteración a través de las frecuencias observadas
for df in frecuencias_observadas:
    num_filas, num_columnas = df.shape
    # Suma el número de filas al total para los grados de libertad
    sum_filas += num_filas
# Calcula los grados de libertad
grados_libertad = (sum_filas - 1) * (num_columnas - 1)

print("Grados de libertad total:", grados_libertad)
# Calcula el valor chi-cuadrado teórico para un nivel de significancia del 5% y los grados de libertad calculados
chiTeorico = chi2.ppf(0.95, grados_libertad)
print("Chi Teorico= ",chiTeorico)

Grados de libertad total: 28
Chi Teorico=  41.33713815142739


In [31]:
# Compara el valor chi-cuadrado observado con el valor chi-cuadrado teórico crítico
if(ChiObs < chiTeorico):
    print("Se acepta la hipótesis nula, por lo tanto se considera estacionario en el tiempo.")
else:
    print("Se rechaza la hipótesis nula, por lo tanto no se considera estacionario en el tiempo.")

Se acepta la hipótesis nula, por lo tanto se considera estacionario en el tiempo.
